<p style="font-size: 12px">En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.</p>

<div hidden>
    <!-- Font normal -->
    <p style="font-size: 16px">text</p>
    <!-- Font pequeño -->
    <p style="font-size: 15px">text</p>
    <!-- hr interno -->
    <hr style="height: 1px; color: hsl(221.25 24.24% 18%); background: hsl(221.25 24.24% 18%); font-size: 0; border: 0">
    <!-- hr externo -->
    <hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">
    <b style="color: hsl(163.3 98.98% 58.43%)">bold</b>
</div>

<h1 align="center">LATAM Data Engineer Challenge</h1>
<hr style="height: 1px; color: hsl(221.25 24.24% 18%); background: hsl(221.25 24.24% 18%); font-size: 0; border: 0">
<p style="font-size: 16px">Hola! gracias por leer! A continuación paso a detallar los pasos realizados y todo lo pensado para este desafio.</p>

<div style="background-color: hsl(221.25 24.24% 18%); width: 30%; padding:5px 10px; border-radius: 5px">
<h3>Tech usadas</h3>
<ul>
    <li>PySpark v3.5.1</li>
    <li>Python v3.9.13</li>
    <li>Pandas v2.2.2</li>
    <li>Java version "1.8.0_381"</li>
</ul>
</div>


<h2>Proceso mental</h2>
<p style="font-size: 16px">Después de revisar la estructura del dataset y revisar la documentación, pensé que lo primero que debía hacer era buscar una manera de unir todos los <b style="color: hsl(163.3 98.98% 58.43%)">quotedTweets</b> ya que finalmente corresponden a otro tweet, y con esto lograr un mayor dataset.</p>

<p style="font-size: 16px">Siguiendo, y como quería usar PySpark creé una clase que instanciara a <b style="color: hsl(163.3 98.98% 58.43%)">Spark</b> para dejar ahí toda la configuración y cualquier método que me fuera útil. Fui probando distintas configuraciones de <b style="color: hsl(163.3 98.98% 58.43%)">Spark</b> según avanzaba en el desafio, y según errores que iba encontrando.</p>

<p style="font-size: 16px">Principalmente sufrí con un par de errores del tipo: "<i>Py4JJavaError: An error occurred while calling {ABC}</i>". Esto ocurría todas las veces que usé UDFs, busqué varias soluciones pero ninguna me sirvió así que opté por no usar UDFs. Pienso que probablemente tengo algo mal seteado en Spark, y por ello me arrojó error cada vez que quise usarlas.</p>

<p style="font-size: 16px">Luego de realizar algunas pruebas con el archivo JSON, preferí pasarlo a Parquet para así mejorar el rendimiento de las ejecuciones.</p>

<p style="font-size: 16px">En cuanto a las respuestas de Tiempo usé <b style="color: hsl(163.3 98.98% 58.43%)">Pandas</b>. Al principio pensé en usar PySpark para todo pero como corría bastante bien en tiempo, pensé que finalmente según se pueda tunear Spark, se puede lograr un buen tiempo de ejecución así como de manejo de memoría. Asi que para variar también mis respuestas me fui por ese camino.</p>

<h2>Mejoras</h2>
<p style="font-size: 16px">Con respecto a las mejoras se me ocurren varias cosas: </p>
<ol>
    <li style="font-size: 16px">Leyendo algunos artículos sobre optimización, creo que se podría utilizar Bucketing, Partitioning y Z-Ordering:</li>
        <ul>
            <li style="font-size: 16px">Partitioning: en el sentido de crear particiones usando alguna columna en particular según se necesite: 'df.write.partitionBy("column_name")'</li>
            <li style="font-size: 16px">Bucketing: con esto se puede agrupar data en "buckets" del mismo tamaño, lo cual ayuda cuando se usa groupBy, joins y orderBy</li>
            <li style="font-size: 16px">Z-Ordering: para ordernar espacialmente la data y juntar data relacionada.</li>
        </ul>
    <li style="font-size: 16px">Reducir aún más el dataset. Para las 3 preguntas solo se necesitaban 5 columnas: "id", "date", "user.username", "mentionedUser[].username" y "content":</li>
        <ul>
            <li style="font-size: 16px">teniendo un dataset más pequeño haría que mejora el rendimiento en el sentido de la carga inicial de la data.</li>
        </ul>
    <li style="font-size: 16px">Trabajar aún mas la columna "content":</li>
        <ul>
            <li style="font-size: 16px">Para la pregunta 2, como finalmente solo se querían contar emojis lo ideal sería solo dejar los carácteres de emojis. La forma que usé para esto fue quitar casi todo pero me quedaron carácteres de lenguajes de la India.</li>
        </ul>
    <li style="font-size: 16px">Pensando en Spark, idea contar con mayor cantidad de clústers, según se necesite. Esto aporta escalabilidad según crezcan los datos.</li>
    <li style="font-size: 16px">Subir todo a la nube:</li>
        <ul>
            <li style="font-size: 16px">Subir los archivos .py a un bucket en GCP</li>
            <li style="font-size: 16px">Luego con el ID de los archivos en el bucket, usarlos para crear Job en Dataproc</li>
            <li style="font-size: 16px">Acá podríamos dejar la data en BigQuery para luego procesarla con alguna herramienta visual</li>
            <li style="font-size: 16px">Todo esto se podría orquestar con Airflow y así monitorear las ejecuciones</li>
        </ul>
</ol>


<br>
<br>
<hr style="height: 1px; color: hsl(221.25 24.24% 18%); background: hsl(221.25 24.24% 18%); font-size: 0; border: 0">

<p style="font-size: 14px">Referencias</p>
<ul>
    <li><a href="https://medium.com/@dipan.saha/getting-started-with-pyspark-day-1-37e5e6fdc14b" style="color: hsl(163.3 98.98% 58.43%); font-size: 14px" target="_blank">Instalar PySpark en windows</a></li>
    <li><a href="https://spark.apache.org/docs/3.0.2/configuration.html" style="color: hsl(163.3 98.98% 58.43%); font-size: 14px" target="_blank">Docs de configuración de PySpark</a></li>
    <li><a href="https://medium.com/@shilpajoshi1635/optimizing-query-performance-in-pyspark-with-partitioning-bucketing-and-z-ordering-a5e215e6c272" style="color: hsl(163.3 98.98% 58.43%); font-size: 14px" target="_blank">Partitioning, Bucketing y Z-ordering</a></li>
    <li><a href="https://www.freecodecamp.org/news/what-is-google-dataproc/" style="color: hsl(163.3 98.98% 58.43%); font-size: 14px" target="_blank">Usemos Dataproc!</a></li>
    <li><a href="https://www.youtube.com/live/R2c6sZYahe0?t=1126s" style="color: hsl(163.3 98.98% 58.43%); font-size: 14px" target="_blank">YT sobre pipelines usando PySpark y Airflow con GCP</a></li>
</ul>

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

<h2>Inicialización de variables</h2>

<p style="font-size: 16px">Creé la celda siguiente para chequear la instalación de Java.</p>

In [1]:
# check de java
from subprocess import check_output, STDOUT
try:
  check_output("java -version", stderr=STDOUT, shell=True).decode('utf-8')
except OSError:
  # raise KeyboardInterrupt("Por favor instale Java JDK para continuar.")
  print("Java no está instalado en su PC, para que el programa funcione correctamente, por favor instale Java en su PC.")

<p style="font-size: 16px">Luego inicializo los nombres de archivos y las rutas, además de preguntar si es que existe el .zip para descomprimirlo en la carpeta <b style="color: hsl(163.3 98.98% 58.43%)">data</b></p>

In [2]:
import os
import cProfile
import pstats
from zipfile import ZipFile

# declaro nombres de archivo y rutas
json_file = "farmers-protest-tweets-2021-2-4.json"
parquet_file = "tweets.parquet"
data_folder_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
json_path = f"{data_folder_path}\\{json_file}"
parquet_path = f"{data_folder_path}\\{parquet_file}"

# max lineas a imprimir en stats
max_print_stat = 100

# checkea si existe la carpeta data, sino, la crea
if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

# checkea si existe el archivo tweets.json.zip, si no, checkea si existe el archivo json, si no, lanza excepción
if not os.path.exists(f"{data_folder_path}\\tweets.json.zip"):
    if not os.path.exists(f"{data_folder_path}\\{json_file}"):
        raise FileNotFoundError("Por favor descargue el archivo 'tweets.json.zip' y colóquelo en la carpeta 'data'")

# si no existe el archivo json, descomprime el archivo zip
if not os.path.exists(f"{data_folder_path}\\{json_file}"):
    with ZipFile(f"{data_folder_path}\\tweets.json.zip", 'r') as zObject: 
        zObject.extractall(path=f"{data_folder_path}")

<p style="font-size: 16px">Luego pregunta si es que existe el archivo parquet, sino, lo crea.</p>

<p style="font-size: 16px">La función <b style="color: hsl(163.3 98.98% 58.43%)">json_to_parquet</b> toma el json principal (el de 389mb) y lo "desnestea". Esto va a buscar si existe algún elemento <b style="color: hsl(163.3 98.98% 58.43%)">quotedTweet</b>, y si existe une sus datos con el dataframe principal, esto para crear un gran dataset con todos los tweets, luego de ellos lo pasa a formato <b style="color: hsl(163.3 98.98% 58.43%)">parquet</b>.</p>

```python
def json_to_parquet(ruta_del_json: str, ruta_del_parquet: str) -> None
```

In [3]:
from helpers.helperFuncs import json_to_parquet
# si no encuentra el archivo parquet, lo crea
if not os.path.exists(f"{parquet_path}"):
    json_to_parquet(json_path, parquet_path)

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

#### Pregunta 1
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns: 
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```

<h4>Notas:</h4>

<p style="font-size: 16px">Creo que las mejoras que se pueden hacer acá es no utilizar "limit", y trabajar sobre las listas resultantes usando "take". Leí que limit se hace en dos pasos, uno local y otro global, este último genera un shuffle lo cual es costoso en dataset grandes. <a href="https://towardsdatascience.com/stop-using-the-limit-clause-wrong-with-spark-646e328774f5" style="color: hsl(163.3 98.98% 58.43%); font-size: 14px" target="_blank">Referencia</a></p>

```python
 # obtengo las top 10 fechas con mas tweets 
top_10_dates = df.groupBy(df["date_only"].alias("date")) \
    .agg(sf.count("id").alias("tweetCount")) \
    .orderBy(sf.desc("tweetCount")) \
    .limit(10).cache()
```

In [4]:
#%%script false --no-raise-error
from q1_memory import q1_memory
q1_mem_result = q1_memory(f"{parquet_path}")
q1_mem_result

Filename: f:\GitHub\DataEngineer\latam_de_challenge\src\q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8     73.2 MiB     73.2 MiB           1   @profile
     9                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    10                                             # Inicializacion de Spark
    11     75.5 MiB      2.3 MiB           1       spark = SparkClass("Q1: Memory")
    12                                             # Carga de datos
    13     75.5 MiB      0.0 MiB           1       df = spark.load_parquet(file_path).select("id", "username", "date")
    14     75.6 MiB      0.1 MiB           3       df = df.withColumn("date_only", sf.col("date").cast(DateType())) \
    15     75.5 MiB      0.0 MiB           1           .repartition(4, "date_only") \
    16     75.5 MiB      0.0 MiB           1           .drop("date").cache()
    17                                         
    18                

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 18), 'rebelpacifist'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 19), 'KaurDosanjh1979')]

<h4>Notas:</h4>

<p style="font-size: 16px">Acá (y en realidad lo mismo para el código de memoria) creo que en vez de crear una nueva columna con la fecha, se podría dejar de antemano en el parquet, para así ahorrar la conversión.</p>

In [5]:
#%%script false --no-raise-error
from q1_time import q1_time

with cProfile.Profile() as pr:
    q1_time_result = q1_time(f"{parquet_path}")

q1_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q1_stats.dump_stats('./stats/q1_time')
q1_stats.print_stats(max_print_stat)
q1_time_result

         937513 function calls (936682 primitive calls) in 1.393 seconds

   Ordered by: internal time
   List reduced from 1132 to 100 due to restriction <100>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    50520    0.358    0.000    0.785    0.000 multi.py:2982(get_loc)
        1    0.247    0.247    0.247    0.247 datetimes.py:456(_array_strptime_with_fallback)
    50549    0.134    0.000    0.134    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   101048    0.096    0.000    0.107    0.000 base.py:3777(get_loc)
    50521    0.085    0.000    0.219    0.000 multi.py:136(_codes_to_ints)
        1    0.069    0.069    0.908    0.908 multi.py:2355(drop)
        1    0.059    0.059    0.060    0.060 pandas_compat.py:755(table_to_dataframe)
        1    0.034    0.034    0.036    0.036 multi.py:758(_values)
        1    0.029    0.029    0.029    0.029 datetimes.py:1436(date)
        3    0.029    0.010    0.030    0.010 algorithms.py:548(factorize_array)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 18), 'rebelpacifist'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 19), 'KaurDosanjh1979')]

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

#### Pregunta 2
Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:
```python
def q2_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("✈️", 6856), ("❤️", 5876), ...]
```

<h4>Notas:</h4>

<p style="font-size: 16px">Como comenté antes, creo que se puede mejorar el patrón de búsqueda de emojis, y el de reducción de la columna "content". Para esta pregunta usé la siguiente expresión regular:</p>

```shell
[\u0900-\u097F]+|[\x00-\x7F]+
```
<p style="font-size: 16px">La cual se encarga de quitar casi todo dejandolo solo emojis, pero si bien funciona bien, creo que se puede mejorar para quitar aún mas carácteres y dejar solo emojis.</p>

<p style="font-size: 14px"><i>Nota aparte: probé este código usando el patrón completo de emojis sacado de la librería "emojis" y me dió el mismo resultado sobre la columna completa sin limpiar.</i></p>


In [6]:
#%%script false --no-raise-error
from q2_memory import q2_memory
q2_mem_result = q2_memory(f"{parquet_path}")
q2_mem_result

Filename: f:\GitHub\DataEngineer\latam_de_challenge\src\q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    11    184.1 MiB    184.1 MiB           1   @profile
    12                                         def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    13                                             # Inicializacion de Spark
    14    184.2 MiB      0.2 MiB           1       spark = SparkClass("Q2: Memory")
    15                                             # Carga de datos
    16    184.2 MiB      0.0 MiB           1       df = spark.load_parquet(file_path).select("content").where(sf.col("content").isNotNull()).cache()
    17                                             
    18                                             # creo patron de emojis
    19                                             # https://carpedm20.github.io/emoji/docs/index.html#regular-expression
    20    184.2 MiB      0.0 MiB           1       emojis = sorted(emoji.EMOJI_DAT

[('🙏', 5538),
 ('😂', 3712),
 ('🚜', 3017),
 ('🌾', 2288),
 ('🇮🇳', 2231),
 ('🤣', 1997),
 ('✊', 1776),
 ('🙏🏻', 1495),
 ('❤️', 1478),
 ('👇', 1118)]

In [7]:
#%%script false --no-raise-error
from q2_time import q2_time

with cProfile.Profile() as pr:
    q2_time_result = q2_time(f"{parquet_path}")

q2_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q2_stats.dump_stats('./stats/q2_time')
q2_stats.print_stats(max_print_stat)
q2_time_result

         5400705 function calls (5400628 primitive calls) in 3.264 seconds

   Ordered by: internal time
   List reduced from 473 to 100 due to restriction <100>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1168115    1.282    0.000    1.889    0.000 tokenizer.py:158(tokenize)
   133454    0.464    0.000    0.464    0.000 {method 'sub' of 're.Pattern' objects}
   133454    0.446    0.000    2.451    0.000 core.py:299(<listcomp>)
  1034680    0.187    0.000    0.189    0.000 {built-in method __new__ of type object at 0x00007FF8B5084C60}
        1    0.120    0.120    0.121    0.121 pandas_compat.py:755(table_to_dataframe)
1035780/1035772    0.117    0.000    0.117    0.000 {built-in method builtins.isinstance}
  1034722    0.103    0.000    0.103    0.000 {method 'append' of 'list' objects}
   133454    0.098    0.000    2.655    0.000 q2_time.py:6(find_all_emojis)
        1    0.081    0.081    0.082    0.082 core.py:1389(read)
   133454    0.080    0.000 

[('🙏', 5538),
 ('😂', 3712),
 ('🚜', 3017),
 ('🌾', 2288),
 ('🇮🇳', 2231),
 ('🤣', 1997),
 ('✊', 1776),
 ('🙏🏻', 1495),
 ('❤️', 1478),
 ('👇', 1118)]

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

#### Pregunta 3
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. Debe incluir las siguientes funciones:
```python
def q3_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("LATAM321", 387), ("LATAM_CHI", 129), ...]
```

<h4>Notas:</h4>

<p style="font-size: 16px">En esta pregunta y como ya tenía reducida la columna "mentionedUsers", se me hizo mas fácil trabajarla. Como mejora, creo que en el parquet, podría agregar una nueva columna que cuente de una las veces que se repite el usuario usando "user.username", de este forma no tendría que pasar por el explode, y simplemente podría sumar este número.</p>

In [8]:
#%%script false --no-raise-error
from q3_memory import q3_memory
q3_mem_result = q3_memory(f"{parquet_path}")
q3_mem_result

Filename: f:\GitHub\DataEngineer\latam_de_challenge\src\q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    244.5 MiB    244.5 MiB           1   @profile
     7                                         def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     8                                             # Inicializacion de Spark
     9    244.6 MiB      0.1 MiB           1       spark = SparkClass("Q3: Memory")
    10                                             # Carga de datos
    11    244.6 MiB      0.0 MiB           1       df = spark.load_parquet(file_path).select("mentionUser").cache()
    12                                         
    13                                             # hago un explode de los mentionedUsers para abrir el array y luego tomo solo el nombre de usuario
    14    244.6 MiB      0.0 MiB           1       df = df.select(sf.explode("mentionUser").alias("username")).select("username")
    15                              

[('narendramodi', 2415),
 ('Kisanektamorcha', 1942),
 ('RakeshTikaitBKU', 1723),
 ('PMOIndia', 1489),
 ('RahulGandhi', 1218),
 ('GretaThunberg', 1105),
 ('RaviSinghKA', 1062),
 ('rihanna', 1037),
 ('UNHumanRights', 989),
 ('meenaharris', 975)]

In [9]:
#%%script false --no-raise-error
from q3_time import q3_time

with cProfile.Profile() as pr:
    q3_time_result = q3_time(f"{parquet_path}")

q3_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q3_stats.dump_stats('./stats/q3_time')
q3_stats.print_stats(max_print_stat)
q3_time_result

         1731 function calls (1694 primitive calls) in 0.095 seconds

   Ordered by: internal time
   List reduced from 370 to 100 due to restriction <100>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.031    0.031    0.032    0.032 series.py:4496(explode)
        1    0.028    0.028    0.029    0.029 pandas_compat.py:755(table_to_dataframe)
        1    0.015    0.015    0.016    0.016 core.py:1389(read)
        1    0.006    0.006    0.006    0.006 {built-in method _collections._count_elements}
        1    0.003    0.003    0.003    0.003 heapq.py:521(nlargest)
        1    0.002    0.002    0.002    0.002 missing.py:305(_isna_string_dtype)
        1    0.002    0.002    0.002    0.002 core.py:1243(__init__)
        1    0.002    0.002    0.002    0.002 {method 'tolist' of 'numpy.ndarray' objects}
        1    0.001    0.001    0.095    0.095 q3_time.py:5(q3_time)
        1    0.001    0.001    0.001    0.001 {method 'repeat' of 'numpy.ndarr

[('narendramodi', 2415),
 ('Kisanektamorcha', 1942),
 ('RakeshTikaitBKU', 1723),
 ('PMOIndia', 1489),
 ('RahulGandhi', 1218),
 ('GretaThunberg', 1105),
 ('RaviSinghKA', 1062),
 ('rihanna', 1037),
 ('UNHumanRights', 989),
 ('meenaharris', 975)]